In [1]:
import numpy as np
from cvxopt.modeling import variable, op
from scipy.optimize import linprog
import warnings 
warnings.filterwarnings("ignore")

# ВАРИАНТ 24

In [2]:
# Запасы
a = np.array([290, 370, 250]) 
#Потребности
b = np.array([140, 130, 210, 270, 160]) 
# Матрица тарифов
C = np.array([[5, 14, 8, 11, 4],
              [15, 10, 12, 13, 10],
              [7, 9, 16, 17, 23]])  

In [3]:
# Функция возвращает матрицу системы ограничений
def ogran(a, b):
    m = len(a)
    n = len(b)
    h = np.diag(np.ones(n))
    v = np.zeros((m, n))
    v[0] = 1
    for i in range(1, m):
        h = np.hstack((h, np.diag(np.ones(n))))
        k = np.zeros((m, n))
        k[i]=1
        v = np.hstack((v, k))
    return np.vstack((h, v)).astype(int), np.hstack((b,a))

# Метод потенциалов
def m_pot(a, b, c):
    # Проверяем замкнутостm:
    if a.sum() > b.sum():
        b = np.hstack((b, [a.sum() - b.sum()]))
        c = np.hstack((c, np.zeros(len(a)).reshape(-1, 1)))
    elif a.sum() < b.sum():
        a = np.hstack((a, [b.sum() - a.sum()]))
        c = np.vstack((c, np.zeros(len(b))))
        
    m = len(a)
    n = len(b)
    A_eq, b_eq = ogran(a, b)
    res = linprog(c.reshape(1, -1), A_eq=A_eq, b_eq=b_eq, bounds=(0, None), method='simplex')
    return res.x.astype(int).reshape(m, n), res.fun.astype(int) 

In [4]:
x2, opt = m_pot(a, b, C)
print('Оптимальный план перевозки груза: \n', x2)
print('Оптимальная стоимость перевозки: ', opt)

Оптимальный план перевозки груза: 
 [[  0   0 130   0 160]
 [  0  20  80 270   0]
 [140 110   0   0   0]]
Оптимальная стоимость перевозки:  8320
